<a href="https://colab.research.google.com/github/helloitsmimi/helloitsmimi/blob/master/%EB%A6%AC%EB%B7%B0_%EA%B8%B0%EB%B0%98_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%97%90%EC%96%B4%ED%94%8C%EB%A1%9C%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

#현재 작업 위치 이동
#띄어쓰기에 \붙일 것
%cd /content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트

Mounted at /content/drive
/content/drive/MyDrive/멀티캠퍼스 파이썬/최종 프로젝트


In [ ]:
# 패키지 설치

import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.models import load_model
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import nltk
import joblib
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def make_data():

  '''
  유저/게임/리뷰 데이터를 이용해
  최종적으로 Doc2Vec에 사용되는 데이터 형태인 TaggedDocument 만들기
  '''

  # 메타 전문가 리뷰 불러오기
  critic_data = pd.read_csv('./data/meta_final/meta_meta_mart.csv')
  # game / review 열로 가져오기, 조건 설정
  d1 = critic_data[(critic_data['platform']=='pc') & (critic_data['criticscore']>=75)][['appid', 'criticcontent']]
  # 이름 각각 game, score으로 변경
  d1.rename(columns = {'appid':'game', 'criticcontent':'review'}, inplace = True)
  # 리뷰 NaN값인 행 제거
  d1.dropna(subset=['review'], inplace=True)



  # 메타 유저 리뷰 불러오기
  user_data = pd.read_csv("./data/meta_final/meta_user_mart.csv")
  # game / review 열로 가져오기, 조건 설정
  d2 = user_data[(user_data['platform']=='pc') & (user_data['userscore']>7.5)][['appid', 'usercontent']]
  # 이름 각각 user, game, score으로 변경
  d2.rename(columns = {'appid':'game', 'usercontent':'review'}, inplace = True)
  # 리뷰 NaN값인 행 제거
  d2.dropna(subset=['review'], inplace=True)



  # 스팀 리뷰 데이터 불러오기
  steam_review = pd.read_csv("./data/steam_data/for_mimi.csv")
  steam_review.dropna(subset=['review'], inplace=True) # 리뷰 NaN값인 행 제거
  steam_review.dropna(subset=['weighted_vote_score'], inplace=True) # weighted_vote_score NaN값인 행 제거
  steam_review.dropna(subset=['voted_up'], inplace=True) # voted_up NaN값인 행 제거
  steam_review.dropna(subset=['appid'], inplace=True) # appid NaN값인 행 제거
  d3 = steam_review[(steam_review['voted_up']==True) & (steam_review['weighted_vote_score']>=0.75)][['appid', 'review']]
  # 이름 각각 game, score으로 변경
  d3.rename(columns = {'appid':'game'}, inplace = True)



  # d1, d2, d3 합치기
  total_data = pd.concat([d1, d2, d3])
  # nan값 있는 행 제거
  total_data.dropna(inplace=True)
  # 게임 명 공백인 행 제거
  drop_index = total_data[total_data['game']==''].index
  total_data.drop(drop_index, axis='index', inplace=True)
  # game열 타입 float에서 int로
  total_data = total_data.astype({'game':'int'})



  # 자연어 처리를 위한 토크나이저 생성
  # 토크나이저의 단어 사전 : 희귀단어 제거하고 생성
  # tokenizer로 total_data['review'] 단어사전 만들고 희귀단어 비율 보기

  tok = Tokenizer() # 토크나이저 할당
  tok.fit_on_texts(total_data['review']) # 리뷰 데이터로 토크나이저 학습

  # 등장 빈도수가 3회 이하 단어를 희귀단어로 취급
  threshold = 3
  total_cnt = len(tok.word_index) # 토크나이저가 생성한 단어사전의 단어 수
  rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
  total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합

  for key, value in tok.word_counts.items():
      total_freq = total_freq + value
      if(value <= threshold):
          rare_cnt = rare_cnt + 1

  # vocab_size : 희귀단어 count 기준
  vocab_size = total_cnt- rare_cnt + 1 



  # 특수문자 제거
  def remain_engnum(sentence):
    sentence = re.sub('[^a-zA-Z0-9]+', ' ', sentence)
    return sentence

  total_data['review'] = total_data['review'].apply(lambda x:remain_engnum(x))

  # 소문자화
  total_data['review']= total_data['review'].apply(lambda x: x.lower())

  # 게임 별로 리뷰 합치기
  game_review = pd.DataFrame(total_data['game'].unique(), columns=['game'])

  for game in total_data['game'].unique():
    game_review.loc[game_review['game']==game, 'review'] = " ".join(total_data[total_data['game']==game]['review'])

  # 희귀단어 제거를 위한 토크나이저 생성
  total_tokenizer = Tokenizer(num_words = vocab_size) # 희귀단어 제거한 단어사전 생성
  total_tokenizer.fit_on_texts(game_review['review']) # 학습

  # 단어사전에서 희귀단어 제거
  words_frequency = [word for word, index in total_tokenizer.word_index.items() if index >= vocab_size + 1]
  for word in words_frequency:
      del total_tokenizer.word_index[word] # 해당 단어에 대한 인덱스 정보를 삭제
      del total_tokenizer.word_counts[word] # 해당 단어에 대한 카운트 정보를 삭제

  # 토큰화
  game_review['review'] = game_review['review'].apply(lambda x:text_to_word_sequence(x))

  # 희귀단어 제거
  def remain_word(review):
    result = []
    for word in review:
      if word in total_tokenizer.word_index.keys():
        result.append(word)
    return result
     
  game_review['review'] = game_review['review'].apply(lambda x:remain_word(x))

  # 불용어 제거
  def remove_stopwords(text): 
    output= [i for i in text if i not in stop_words] 
    return output
    
  stop_words = set(stopwords.words('english'))
  game_review['review'] = game_review['review'].apply(lambda x:remove_stopwords(x))

  doc_df = game_review[['game','review']].values.tolist()
  tagged_data = [TaggedDocument(words=_d, tags=[gid]) for gid, _d in doc_df]

  return tagged_data

In [ ]:
# Doc2Vec

def make_model(tagged_data):

  '''
  make_data()로 생성한 tagged_data를 사용해서
  Doc2Vec으로 리뷰 간 유사도 계산

  make_data()를 먼저 변수에 할당하고
  make_model()에 매개변수로 입력
  '''

  # Doc2Vec 학습
  max_epochs = 10 # epoch 10번

  model = Doc2Vec(
    window=5, # 모델 학습 시 앞 뒤로 보는 단어 개수
    vector_size=100, # 벡터차원 크기
    alpha=0.025, # learning rate
    min_alpha=0.025,# min learning rate. 학습 진행될수록 alpha값이 min_alpha로 근사
    min_count=3, # 학습에 사용할 최소 단어 빈도 수
    dm =1, # 학습 방법 : 1-pvdm, 0-pvdbow
    negative = 5, # complexity reudction. negative sampling 시 샘플링 할 단어 개수 지정
    seed = 9999)

  model.build_vocab(tagged_data) # 단어사전 생성

  for epoch in range(max_epochs): # tagged_data로 10번 학습
      print('iteration {0}'.format(epoch))
      model.train(tagged_data, # 학습에 사용할 데이터
                  total_examples=model.corpus_count,
                  epochs=model.epochs)
     # decrease the learning rate
      model.alpha -= 0.002
     # fix the learning rate, no decay
      model.min_alpha = model.alpha

  model.save('./model/finaldatadoc2vec.model') # 수정

In [ ]:
model = Doc2Vec.load('./model/finaldatadoc2vec.model') # make_model()에서 저장했던 모델 불러오기

In [ ]:
def gentleman_ver1(game1, game2, game3):

  '''
  추천 게임 출력 함수
  '''


  d1 = pd.DataFrame(model.docvecs.most_similar([game1], topn=10), columns=['game', 'similarity1']) # game1 추천목록 & 유사도
  d2 = pd.DataFrame(model.docvecs.most_similar([game2], topn=10), columns=['game', 'similarity2']) # game2 추천목록 & 유사도
  d3 = pd.DataFrame(model.docvecs.most_similar([game3], topn=10), columns=['game', 'similarity3']) # game3 추천목록 & 유사도
  df = pd.merge(d1, d2, on='game', how='outer') # d1, d2 merge
  df = pd.merge(df, d3, on='game', how='outer') # d2, d3 merge

  for game in df['game']: # ['game'] 행 하나씩 돌면서 
    if game in [game1, game2, game3]: # game이 game1, game2,m game3에 해당하는 경우
      drop_index = df[df['game']==game].index
      df.drop(drop_index, axis='index', inplace=True) # 해당 행 drop

  # 추천 리스트 간 중복되는 게임이 없는 경우 : 각 추천리스트에서 2개씩 추출
  if len(df) == 30: 
    return [x[0] for x in model.docvecs.most_similar([game1], topn=2)] + [x[0] for x in model.docvecs.most_similar([game2], topn=2)] + [x[0] for x in model.docvecs.most_similar([game3], topn=2)]
  # 추천 리스트 간 중복되는 게임 있는 경우 : 유사도 평균 내서 상위 6개 추출
  else : 
    df.fillna(0, inplace=True)
    df['mean'] = df.mean(axis=1, numeric_only=True)
    df.sort_values('mean', ascending=False, inplace=True)
    return list(df['game'])[:6]

In [ ]:
gentleman_ver1(1445990, 728880, 546560)

[1915840, 387020, 448510, 1097770, 70, 220]